In [1]:
import datetime
import time
import random
import requests
import json
import bson
import gzip
import os

LOCAL = 'http://localhost:8000/'
REMOTE = 'https://api-recintos.herokuapp.com/'
HOME = LOCAL
CARGA = 100

In [2]:
letras = 'ABCDEFGHIJKLMNOPQRSTUVXZ'
numeros = ''.join([str(i) for i in range(10)])
textos = 'ABCDEFGHIJKLMNOPQRSTUVXZ          abcdefghijklmnopqrstuvwxyz'
def random_str(num=3, fila=letras):
    result = ''
    for i in range(num):
        result += random.choice(fila)
    return result

placas = [random_str(3) + random_str(5, numeros) for i in range(100)]
reboques = [random_str(3) + random_str(5, numeros) for i in range(200)]
conteineres = [random_str(4) + random_str(7, numeros) for i in range(200)]
operadores = [random_str(11) for i in range(10)]
motoristas = [random_str(11) for i in range(50)]
textos = [random_str(random.randint(10, 20), textos) for i in range(50)]

In [3]:
pesagens = []
for r in range(CARGA):
    data = datetime.datetime.now().isoformat()
    operador = random.choice(operadores)
    conteiner = random.choice(conteineres)
    tara = random.randint(9000, 12000)
    pesobrutodeclarado = random.randint(3000, 15000)
    pesobalanca = tara + random.randint(-2000, 2000)
    placa = random.choice(placas)
    reboque = random.choice(reboques)
    texto = random.choice(textos)
    pesagem = \
    {'IDEvento': r,
     'capturaautomatica': True,
     'conteiner': random.choice(conteineres),
     'dataevento': data,
     'dataregistro': data,
     'retificador': False,
     'documentotransporte': texto,
     'operadorevento': operador,
     'operadorregistro': operador,
     'pesobalanca': pesobalanca,
     'pesobrutodeclarado': pesobrutodeclarado,
     'placa': placa,
     'placasemireboque': reboque,
     'taraconjunto': tara,
     'tipodocumentotransporte': 'CE'}
    pesagens.append(pesagem)


In [4]:
pesagens[:2]

[{'IDEvento': 0,
  'capturaautomatica': True,
  'conteiner': 'BUBF8943249',
  'dataevento': '2019-06-06T17:01:32.552035',
  'dataregistro': '2019-06-06T17:01:32.552035',
  'documentotransporte': 'fzNIXPKp XaUp BJg N',
  'operadorevento': 'DEQPJKQFBIQ',
  'operadorregistro': 'DEQPJKQFBIQ',
  'pesobalanca': 9831,
  'pesobrutodeclarado': 7822,
  'placa': 'GJB18024',
  'placasemireboque': 'UKE70038',
  'retificador': False,
  'taraconjunto': 9500,
  'tipodocumentotransporte': 'CE'},
 {'IDEvento': 1,
  'capturaautomatica': True,
  'conteiner': 'CRUE8514852',
  'dataevento': '2019-06-06T17:01:32.552130',
  'dataregistro': '2019-06-06T17:01:32.552130',
  'documentotransporte': 'nnmS BmU LjGnB',
  'operadorevento': 'CSHOHLUUQZO',
  'operadorregistro': 'CSHOHLUUQZO',
  'pesobalanca': 9010,
  'pesobrutodeclarado': 10021,
  'placa': 'TUE34705',
  'placasemireboque': 'UJL87623',
  'retificador': False,
  'taraconjunto': 9971,
  'tipodocumentotransporte': 'CE'}]

In [5]:
s0 = time.time()
for pesagem in pesagens[:CARGA // 2]:
    rv = requests.post(HOME + 'pesagemmaritimo',
                   json=pesagem)
    print(rv.status_code)
    print(rv.text)
s1 = time.time()

201
{
  "detail": -2630820615434521092,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -6985564727212786740,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -3879330849201204921,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -8021578233951228522,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": 2633029649711913150,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -4840639057439225223,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -8081215139608642871,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": 1670582431336746153,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": 8827964910282698053,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": 5371208353120432597,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -4208173863372055800,
  "status": 201,
  "title": "Evento incluido"
}

201
{
  "detail": -823761

In [6]:
print('Gravados %d registros em %d segundos' % (CARGA // 2, s1-s0) )
print((s1 - s0) / (CARGA // 2))

Gravados 50 registros em 2 segundos
0.05366659641265869


In [7]:
s0 = time.time()
lista_eventos = []
for IDEvento in range(CARGA):
    rv = requests.get(HOME + 'pesagemmaritimo/' + str(IDEvento))
    if rv.status_code == 200:
        lista_eventos.append((rv.status_code, 
                              rv.json()))
#    print(rv.json())
s1 = time.time()

In [8]:
print('Consultados %d registros em %s segundos' % (len(lista_eventos), s1-s0))
print((s1 - s0) / len(lista_eventos))

Consultados 50 registros em 0.45015764236450195 segundos
0.009003152847290039


In [9]:
lista_eventos[:2]

[(200,
  {'ID': 1,
   'IDEvento': 0,
   'capturaautomatica': True,
   'dataevento': '2019-06-06T17:01:32.552035Z',
   'dataregistro': '2019-06-06T17:01:32.552035Z',
   'documentotransporte': 'fzNIXPKp XaUp BJg N',
   'hash': -2630820615434521092,
   'numero': None,
   'operadorevento': 'DEQPJKQFBIQ',
   'operadorregistro': 'DEQPJKQFBIQ',
   'pesobalanca': 9831,
   'pesobrutodeclarado': 7822,
   'placa': 'GJB18024',
   'placasemireboque': 'UKE70038',
   'recinto': '00001',
   'request_IP': '127.0.0.1',
   'retificador': False,
   'taraconjunto': 9500,
   'time_created': '2019-06-06T20:01:32.635819Z',
   'tipodocumentotransporte': 'CE'}),
 (200,
  {'ID': 2,
   'IDEvento': 1,
   'capturaautomatica': True,
   'dataevento': '2019-06-06T17:01:32.552130Z',
   'dataregistro': '2019-06-06T17:01:32.552130Z',
   'documentotransporte': 'nnmS BmU LjGnB',
   'hash': -6985564727212786740,
   'numero': None,
   'operadorevento': 'CSHOHLUUQZO',
   'operadorregistro': 'CSHOHLUUQZO',
   'pesobalanca': 90

## Teste de tamanho de arquivos em formato json, json .gzip, bson e bson .gzip

In [10]:
def tofile(lista, newfilename, zipped=False):
    """Salva instância em arquivo (padrão BSON)."""
    with open(newfilename, 'wb') as f:
        payload = bson.BSON.encode(lista)
        if zipped:
            payload = gzip.compress(payload)
        f.write(payload)

json_pesagens = {'PesagemMaritimo': pesagens}

with open('test.json', 'w', encoding='utf-8', newline='') as json_out:
    json.dump(json_pesagens, json_out)
size_json = os.path.getsize('test.json')

with open('test.json.gzip', 'wb') as json_out:
    payload = json.dumps(json_pesagens)
    payload = gzip.compress(payload.encode('utf-8'))
    json_out.write(payload)
size_jsongzip = os.path.getsize('test.json.gzip')

tofile(json_pesagens, 'test.bson')
size_bson = os.path.getsize('test.bson')

tofile(json_pesagens, 'test.bson.gzip', zipped=True)
size_bsongzip = os.path.getsize('test.bson.gzip')


In [11]:
print('size_json: %d, %d%s' % (size_json, (size_json/size_json)*100, '%'))
print('size_json_gzip: %d, %d%s ' % (size_jsongzip, (size_jsongzip/size_json)*100, '%'))
print('size_bson: %d, %d%s ' % (size_bson, (size_bson/size_json)*100, '%'))
print('size_bson_gzip: %d, %d%s ' % (size_bsongzip, (size_bsongzip/size_json)*100, '%'))

size_json: 45427, 100%
size_json_gzip: 5910, 13% 
size_bson: 42172, 92% 
size_bson_gzip: 6361, 14% 


In [12]:
s0 = time.time()
query = {'IDEvento': 0,
       'tipoevento': 'PesagemMaritimo'}
r = requests.post(HOME + 'eventosnovos/list',
                  json=query)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 2, 
    "IDEvento": 1, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.552130Z", 
    "dataregistro": "2019-06-06T17:01:32.552130Z", 
    "documentotransporte": "nnmS BmU LjGnB", 
    "hash": -6985564727212786740, 
    "numero": null, 
    "operadorevento": "CSHOHLUUQZO", 
    "operadorregistro": "CSHOHLUUQZO", 
    "pesobalanca": 9010, 
    "pesobrutodeclarado": 10021, 
    "placa": "TUE34705", 
    "placasemireboque": "UJL87623", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 9971, 
    "time_created": "2019-06-06T20:01:32.751720Z", 
    "tipodocumentotransporte": "CE"
  }, 
  {
    "ID": 3, 
    "IDEvento": 2, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.552181Z", 
    "dataregistro": "2019-06-06T17:01:32.552181Z", 
    "documentotransporte": "JJMV uOZEeQnAl", 
    "hash": -3879330849201204921, 
    "numero": null, 
    "operadorevento": "ENMMMJGGSUE", 
  

In [13]:
lista_eventos = json.loads(r.text)

In [14]:
print('Consultados %d registros em %s segundos' % (len(lista_eventos), s1-s0))
print((s1 - s0) / len(lista_eventos))

Consultados 49 registros em 0.022367477416992188 segundos
0.00045647913095902426


In [15]:
lista_eventos[:10]

[{'ID': 2,
  'IDEvento': 1,
  'capturaautomatica': True,
  'dataevento': '2019-06-06T17:01:32.552130Z',
  'dataregistro': '2019-06-06T17:01:32.552130Z',
  'documentotransporte': 'nnmS BmU LjGnB',
  'hash': -6985564727212786740,
  'numero': None,
  'operadorevento': 'CSHOHLUUQZO',
  'operadorregistro': 'CSHOHLUUQZO',
  'pesobalanca': 9010,
  'pesobrutodeclarado': 10021,
  'placa': 'TUE34705',
  'placasemireboque': 'UJL87623',
  'recinto': '00001',
  'request_IP': '127.0.0.1',
  'retificador': False,
  'taraconjunto': 9971,
  'time_created': '2019-06-06T20:01:32.751720Z',
  'tipodocumentotransporte': 'CE'},
 {'ID': 3,
  'IDEvento': 2,
  'capturaautomatica': True,
  'dataevento': '2019-06-06T17:01:32.552181Z',
  'dataregistro': '2019-06-06T17:01:32.552181Z',
  'documentotransporte': 'JJMV uOZEeQnAl',
  'hash': -3879330849201204921,
  'numero': None,
  'operadorevento': 'ENMMMJGGSUE',
  'operadorregistro': 'ENMMMJGGSUE',
  'pesobalanca': 9361,
  'pesobrutodeclarado': 7501,
  'placa': 'MRB6

In [16]:
json_pesagens_teste = {'PesagemMaritimo': pesagens[50:]}

with open('test2.json', 'w', encoding='utf-8', newline='') as json_out:
    json.dump(json_pesagens_teste, json_out)

s0 = time.time()
files = {'file': ('test2.json', open('test2.json', 'r'), 'text/json')}
headers = {}
# headers['Content-Type'] = 'image/jpeg'
r = requests.post(HOME + 'eventosnovos/upload',
                  files=files)
s1 = time.time()
print(r.status_code)
print(r.text)


201
[
  {
    "IDEvento": 50, 
    "hash": 8850474689305281918
  }, 
  {
    "IDEvento": 51, 
    "hash": -8550123570155993018
  }, 
  {
    "IDEvento": 52, 
    "hash": -6455069512928741112
  }, 
  {
    "IDEvento": 53, 
    "hash": 3941752117971284475
  }, 
  {
    "IDEvento": 54, 
    "hash": -4873016251598876625
  }, 
  {
    "IDEvento": 55, 
    "hash": -6752058954538184183
  }, 
  {
    "IDEvento": 56, 
    "hash": -4732568195499980895
  }, 
  {
    "IDEvento": 57, 
    "hash": 2972308974534991938
  }, 
  {
    "IDEvento": 58, 
    "hash": 1940440668299316170
  }, 
  {
    "IDEvento": 59, 
    "hash": -4635513391465201253
  }, 
  {
    "IDEvento": 60, 
    "hash": 478259164985860024
  }, 
  {
    "IDEvento": 61, 
    "hash": 5147801321506632031
  }, 
  {
    "IDEvento": 62, 
    "hash": -198638111487454798
  }, 
  {
    "IDEvento": 63, 
    "hash": 3024816659977401269
  }, 
  {
    "IDEvento": 64, 
    "hash": 404666056294917152
  }, 
  {
    "IDEvento": 65, 
    "hash": 48317669

In [17]:
print('Gravados %d registros em %s segundos' % (CARGA // 2, s1-s0))
print((s1 - s0) / (CARGA // 2))

Gravados 50 registros em 0.15436434745788574 segundos
0.003087286949157715


In [18]:
s0 = time.time()
query = {'IDEvento': 50,
       'tipoevento': 'PesagemMaritimo'}
r = requests.get(HOME + 'eventosnovos/get',
                  data=query)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 52, 
    "IDEvento": 51, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.554287Z", 
    "dataregistro": "2019-06-06T17:01:32.554287Z", 
    "documentotransporte": "F Z  OXVTcH", 
    "hash": -8550123570155993018, 
    "numero": null, 
    "operadorevento": "VJDVXZSVJCD", 
    "operadorregistro": "VJDVXZSVJCD", 
    "pesobalanca": 10694, 
    "pesobrutodeclarado": 8216, 
    "placa": "ZIC08277", 
    "placasemireboque": "AVN99995", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 10356, 
    "time_created": "2019-06-06T20:01:36.062325Z", 
    "tipodocumentotransporte": "CE"
  }, 
  {
    "ID": 53, 
    "IDEvento": 52, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.554328Z", 
    "dataregistro": "2019-06-06T17:01:32.554328Z", 
    "documentotransporte": "spzxtLXRMGuJG ", 
    "hash": -6455069512928741112, 
    "numero": null, 
    "operadorevento": "SVHIBCTEMRS", 


In [19]:
s0 = time.time()
query = {'IDEvento': 50,
       'tipoevento': 'PesagemMaritimo'}
r = requests.post(HOME + 'eventosnovos/list',
                  json=query)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 52, 
    "IDEvento": 51, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.554287Z", 
    "dataregistro": "2019-06-06T17:01:32.554287Z", 
    "documentotransporte": "F Z  OXVTcH", 
    "hash": -8550123570155993018, 
    "numero": null, 
    "operadorevento": "VJDVXZSVJCD", 
    "operadorregistro": "VJDVXZSVJCD", 
    "pesobalanca": 10694, 
    "pesobrutodeclarado": 8216, 
    "placa": "ZIC08277", 
    "placasemireboque": "AVN99995", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 10356, 
    "time_created": "2019-06-06T20:01:36.062325Z", 
    "tipodocumentotransporte": "CE"
  }, 
  {
    "ID": 53, 
    "IDEvento": 52, 
    "capturaautomatica": true, 
    "dataevento": "2019-06-06T17:01:32.554328Z", 
    "dataregistro": "2019-06-06T17:01:32.554328Z", 
    "documentotransporte": "spzxtLXRMGuJG ", 
    "hash": -6455069512928741112, 
    "numero": null, 
    "operadorevento": "SVHIBCTEMRS", 
